In [158]:
import pandas as pd
import requests
import json
from pprint import pprint
from math import radians
import numpy as np
# Retrieve Google API key from config.py
from config import gkey
import math

In [159]:
data_one_path = 'Resources/airbnbmark1.csv'
data_two_path = 'Resources/AB_NYC_2019.csv'
data_one = pd.read_csv(data_one_path)
data_two = pd.read_csv(data_two_path)

C:\Users\Charles Hilgeman\anaconda\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [160]:
data_one.head(1)

,id,name,summary,host_id,host_response_rate,host_acceptance_rate,host_is_superhost,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,...,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,2595,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",2845,50%,38%,f,Midtown,Midtown,Manhattan,...,365,48,94.0,9.0,9.0,10.0,10.0,10.0,9.0,0.39


In [161]:
ident = data_one['id']
name = data_one['name']
rpm = data_one['reviews_per_month']
peoples = data_one['accommodates']
cleaning = data_one['cleaning_fee']
avg_rating = data_one['review_scores_rating']
accuracy = data_one['review_scores_rating']
cleanliness = data_one['review_scores_rating']
checkin = data_one['review_scores_checkin']
comm = data_one['review_scores_communication']
location = data_one['review_scores_location']
value = data_one['review_scores_value']

basic_info_df = pd.DataFrame({'ID': ident,
                             'Room Name': name,
                              'Accomodates': peoples,
                              'Cleaning Fee': cleaning,
                             'Reviews Per Month': rpm,
                             'Rating Score': avg_rating,
                             'Accuracy Score': accuracy,
                             'Cleanliness': cleanliness,
                             'Checkin Score': checkin,
                             'Communication Score': comm,
                             'Location Score': location,
                             'Value Score': value})
basic_info_df.head(2)

,ID,Room Name,Accomodates,Cleaning Fee,Reviews Per Month,Rating Score,Accuracy Score,Cleanliness,Checkin Score,Communication Score,Location Score,Value Score
0,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,10.0,10.0,9.0
1,3831,Cozy Entire Floor of Brownstone,3,NaN,4.69,90.0,90.0,90.0,10.0,9.0,10.0,9.0


In [162]:
ident2 = data_two['id']
host_id = data_two['host_id']
borough = data_two['neighbourhood_group']
neighborhood = data_two['neighbourhood']
lat = data_two['latitude']
lng = data_two['longitude']
room_type = data_two['room_type']
price = data_two['price']
review_num = data_two['number_of_reviews']
avail = data_two['availability_365']

In [163]:
basic_two_df = pd.DataFrame({'ID': ident2,
                            'Host ID': host_id,
                            'Borough': borough,
                            'Neighbourhood': neighborhood,
                            'Latitude': lat,
                            'Longitude': lng,
                            'Type Room': room_type,
                            'Price (per night)': price,
                            'Number of Reviews': review_num,
                            'Availability per 365': avail})
basic_two_df

,ID,Host ID,Borough,Neighbourhood,Latitude,Longitude,Type Room,Price (per night),Number of Reviews,Availability per 365
0,2539,2787,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,9,365
1,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,45,355
2,3647,4632,Manhattan,Harlem,40.80902,-73.94190,Private room,150,0,365
3,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,270,194
4,5022,7192,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,9,0
...,...,...,...,...,...,...,...,...,...,...
48890,36484665,8232441,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,0,9
48891,36485057,6570630,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,0,36
48892,36485431,23492952,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,0,27
48893,36485609,30985759,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,0,2


In [164]:
combined_data = pd.merge(basic_info_df, basic_two_df, on = 'ID')
cleaned_data = combined_data.dropna(how = 'any')
cleaned_data

,ID,Room Name,Accomodates,Cleaning Fee,Reviews Per Month,Rating Score,Accuracy Score,Cleanliness,Checkin Score,Communication Score,...,Value Score,Host ID,Borough,Neighbourhood,Latitude,Longitude,Type Room,Price (per night),Number of Reviews,Availability per 365
0,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,10.0,...,9.0,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,45,355
1,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,10.0,...,9.0,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,45,355
2,2595,Skylit Midtown Castle,2,$95.00,0.38,94.0,94.0,94.0,10.0,10.0,...,9.0,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,45,355
6,5099,Large Cozy 1 BR Apartment In Midtown East,2,$125.00,0.59,90.0,90.0,90.0,10.0,10.0,...,9.0,7322,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,74,129
7,5099,Large Cozy 1 BR Apartment In Midtown East,2,$125.00,0.60,90.0,90.0,90.0,10.0,10.0,...,9.0,7322,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,74,129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113091,35596905,CONEY ISLAND BEACH close to Manhattan New York,4,$100.00,0.28,100.0,100.0,100.0,10.0,10.0,...,10.0,29622548,Brooklyn,Coney Island,40.58101,-73.98514,Entire home/apt,149,0,297
113092,35706482,Trendy Williamsburg Apartment,2,$50.00,0.13,100.0,100.0,100.0,10.0,10.0,...,10.0,16342788,Brooklyn,Williamsburg,40.70690,-73.94521,Private room,79,0,342
113093,35948011,"Classic, Urban with a splash of Glam! Near M...",3,$119.00,2.42,97.0,97.0,97.0,10.0,10.0,...,10.0,269165235,Manhattan,East Village,40.72546,-73.98703,Entire home/apt,235,1,326
113094,36176586,BRAND NEW KING 2br Airports& Times Sq MINUTES ...,10,$80.00,2.50,85.0,85.0,85.0,9.0,8.0,...,9.0,272080566,Queens,Elmhurst,40.73480,-73.86906,Entire home/apt,199,0,157


In [165]:
# cleaned_data.to_csv('Resources/cleaned_data.csv')

In [166]:
#Find location for Manhattan, Brooklyn, and Bronx
boroughs_df = pd.read_csv("Resources/Boroughs.csv")
boroughs_df.head()


,Boroughs
0,The Bronx
1,Brooklyn
2,Manhattan


In [167]:
boroughs_df["borough lng"]= ""
boroughs_df["borough lat"]= ""
boroughs_df["attraction name"]= ""
boroughs_df["attraction lng"]= ""
boroughs_df["attraction lat"]= ""
boroughs_df

,Boroughs,borough lng,borough lat,attraction name,attraction lng,attraction lat
0,The Bronx,,,,,
1,Brooklyn,,,,,
2,Manhattan,,,,,


In [168]:
params = {"key": gkey}

for index, row in boroughs_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['Boroughs']
    state = 'New York'

    # update address key value
    params['address'] = f"{city},{state}"

    # make request
    boroughs_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
#     print(cities_lat_lng.url)
    
#     # convert to json
    boroughs_lat_lng = boroughs_lat_lng.json()

    boroughs_df.loc[index, "borough lat"] = boroughs_lat_lng["results"][0]["geometry"]["location"]["lat"]
    boroughs_df.loc[index, "borough lng"] = boroughs_lat_lng["results"][0]["geometry"]["location"]["lng"]

# # Visualize to confirm lat lng appear
boroughs_df.head()

,Boroughs,borough lng,borough lat,attraction name,attraction lng,attraction lat
0,The Bronx,-73.864827,40.844782,,,
1,Brooklyn,-73.944158,40.678178,,,
2,Manhattan,-73.971249,40.78306,,,


In [169]:
#Get locations for 1 attraction in each borough
params = {
    "radius": 10000,
    "types": "tourist_attraction",
    "key": gkey
}

for index, row in boroughs_df.iterrows():
    # get lat, lng from df
    lat = row["borough lat"]
    lng = row["borough lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    attraction = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(attraction.url)

    # convert to json
    attraction = attraction.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        boroughs_df.loc[index, "attraction name"] = attraction["results"][0]["name"]
        boroughs_df.loc[index, "attraction lng"] = attraction["results"][0]["geometry"]["location"]["lng"]
        boroughs_df.loc[index, "attraction lat"] = attraction["results"][0]["geometry"]["location"]["lat"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
boroughs_df

,Boroughs,borough lng,borough lat,attraction name,attraction lng,attraction lat
0,The Bronx,-73.864827,40.844782,Bronx Zoo,-73.876998,40.850595
1,Brooklyn,-73.944158,40.678178,Brooklyn Children's Museum,-73.943986,40.674461
2,Manhattan,-73.971249,40.78306,American Museum of Natural History,-73.973988,40.781324


In [170]:
boroughs_df = boroughs_df.rename(columns={'Boroughs':'Borough'})
boroughs_df

,Borough,borough lng,borough lat,attraction name,attraction lng,attraction lat
0,The Bronx,-73.864827,40.844782,Bronx Zoo,-73.876998,40.850595
1,Brooklyn,-73.944158,40.678178,Brooklyn Children's Museum,-73.943986,40.674461
2,Manhattan,-73.971249,40.78306,American Museum of Natural History,-73.973988,40.781324


In [171]:
boroughs_df =boroughs_df.set_index('Borough')
boroughs_df

,borough lng,borough lat,attraction name,attraction lng,attraction lat
Borough,,,,,
The Bronx,-73.864827,40.844782,Bronx Zoo,-73.876998,40.850595
Brooklyn,-73.944158,40.678178,Brooklyn Children's Museum,-73.943986,40.674461
Manhattan,-73.971249,40.78306,American Museum of Natural History,-73.973988,40.781324


In [172]:
boroughs_final = boroughs_df.rename(index={'The Bronx':'Bronx'})
boroughs_final

,borough lng,borough lat,attraction name,attraction lng,attraction lat
Borough,,,,,
Bronx,-73.864827,40.844782,Bronx Zoo,-73.876998,40.850595
Brooklyn,-73.944158,40.678178,Brooklyn Children's Museum,-73.943986,40.674461
Manhattan,-73.971249,40.78306,American Museum of Natural History,-73.973988,40.781324


In [173]:
boroughs_final = boroughs_final.reset_index()
boroughs_final

,Borough,borough lng,borough lat,attraction name,attraction lng,attraction lat
0,Bronx,-73.864827,40.844782,Bronx Zoo,-73.876998,40.850595
1,Brooklyn,-73.944158,40.678178,Brooklyn Children's Museum,-73.943986,40.674461
2,Manhattan,-73.971249,40.78306,American Museum of Natural History,-73.973988,40.781324


In [174]:
bnb_attr_merge = pd.merge(cleaned_data, boroughs_final, on = "Borough", how = 'outer')
bnb_attr_merge.head()

,ID,Room Name,Accomodates,Cleaning Fee,Reviews Per Month,Rating Score,Accuracy Score,Cleanliness,Checkin Score,Communication Score,...,Longitude,Type Room,Price (per night),Number of Reviews,Availability per 365,borough lng,borough lat,attraction name,attraction lng,attraction lat
0,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,10.0,...,-73.98377,Entire home/apt,225,45,355,-73.971249,40.78306,American Museum of Natural History,-73.973988,40.781324
1,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,10.0,...,-73.98377,Entire home/apt,225,45,355,-73.971249,40.78306,American Museum of Natural History,-73.973988,40.781324
2,2595,Skylit Midtown Castle,2,$95.00,0.38,94.0,94.0,94.0,10.0,10.0,...,-73.98377,Entire home/apt,225,45,355,-73.971249,40.78306,American Museum of Natural History,-73.973988,40.781324
3,5099,Large Cozy 1 BR Apartment In Midtown East,2,$125.00,0.59,90.0,90.0,90.0,10.0,10.0,...,-73.97500,Entire home/apt,200,74,129,-73.971249,40.78306,American Museum of Natural History,-73.973988,40.781324
4,5099,Large Cozy 1 BR Apartment In Midtown East,2,$125.00,0.60,90.0,90.0,90.0,10.0,10.0,...,-73.97500,Entire home/apt,200,74,129,-73.971249,40.78306,American Museum of Natural History,-73.973988,40.781324


In [175]:
#drop columns that are not needed in the merged dataset
almost_final_df = bnb_attr_merge.drop(columns= ['borough lng','borough lat'])

In [176]:
close_final_df = almost_final_df.set_index('Borough')

In [177]:
close_final_df = close_final_df.drop(['Queens', 'Staten Island'])

In [178]:
close_final_df = close_final_df.reset_index()
close_final_df = close_final_df.drop_duplicates(subset = ['Room Name'])

In [179]:
close_final_df = close_final_df.reset_index().drop(columns = ['index'])

In [180]:
close_coord_df = close_final_df[['Latitude','Longitude','attraction lat','attraction lng']]
close_coord_df

,Latitude,Longitude,attraction lat,attraction lng
0,40.75362,-73.98377,40.781324,-73.973988
1,40.74767,-73.97500,40.781324,-73.973988
2,40.76489,-73.98493,40.781324,-73.973988
3,40.71344,-73.99037,40.781324,-73.973988
4,40.76076,-73.98867,40.781324,-73.973988
...,...,...,...,...
24873,40.87852,-73.85003,40.850595,-73.876998
24874,40.84328,-73.88173,40.850595,-73.876998
24875,40.86259,-73.90359,40.850595,-73.876998
24876,40.84321,-73.83129,40.850595,-73.876998


In [181]:
close_coord_df['Latitude Distance'] = close_coord_df['Latitude'] - close_coord_df['attraction lat']
close_coord_df['Longitude Distance'] = close_coord_df['Longitude'] - close_coord_df['attraction lng']
close_coord_df

<ipython-input-181-71894d0c4805>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_coord_df['Latitude Distance'] = close_coord_df['Latitude'] - close_coord_df['attraction lat']
<ipython-input-181-71894d0c4805>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_coord_df['Longitude Distance'] = close_coord_df['Longitude'] - close_coord_df['attraction lng']


,Latitude,Longitude,attraction lat,attraction lng,Latitude Distance,Longitude Distance
0,40.75362,-73.98377,40.781324,-73.973988,-0.027704,-0.009782
1,40.74767,-73.97500,40.781324,-73.973988,-0.033654,-0.001012
2,40.76489,-73.98493,40.781324,-73.973988,-0.016434,-0.010942
3,40.71344,-73.99037,40.781324,-73.973988,-0.067884,-0.016382
4,40.76076,-73.98867,40.781324,-73.973988,-0.020564,-0.014682
...,...,...,...,...,...,...
24873,40.87852,-73.85003,40.850595,-73.876998,0.027925,0.026968
24874,40.84328,-73.88173,40.850595,-73.876998,-0.007315,-0.004732
24875,40.86259,-73.90359,40.850595,-73.876998,0.011995,-0.026592
24876,40.84321,-73.83129,40.850595,-73.876998,-0.007385,0.045708


In [182]:
a_mid = []
for dist in close_coord_df['Latitude Distance']:
    a_mid.append(math.sin(dist/ 2)**2)

In [183]:
b_start = []
for locs in close_coord_df['Latitude']:
    b_start.append(math.cos(locs))

In [184]:
b_mid = []
for locas in close_coord_df['attraction lat']:
    b_mid.append(math.cos(locs))

In [185]:
b_fin = []
for distas in close_coord_df['Longitude Distance']:
    b_fin.append(math.sin(distas / 2)**2)

In [186]:
geo_coords_df = pd.DataFrame({'a_mid': a_mid,
                             'b_start': b_start,
                             'b_mid': b_mid,
                             'b_fin': b_fin})

In [187]:
geo_coords_df['b'] = geo_coords_df['b_start'] * geo_coords_df['b_mid'] * geo_coords_df['b_fin']
geo_coords_df['a'] = geo_coords_df['b'] + geo_coords_df['a_mid']

In [188]:
c = []
for number in geo_coords_df['a']:
    c.append(2*math.atan2(math.sqrt(number),math.sqrt(1-number)))
geo_coords_df['c'] = c
geo_coords_df['Proximity'] = geo_coords_df['c']*52.80

In [189]:
final_df = close_final_df
final_df['Proximity (mi)'] = geo_coords_df['Proximity']

In [190]:
final_df

,Borough,ID,Room Name,Accomodates,Cleaning Fee,Reviews Per Month,Rating Score,Accuracy Score,Cleanliness,Checkin Score,...,Latitude,Longitude,Type Room,Price (per night),Number of Reviews,Availability per 365,attraction name,attraction lng,attraction lat,Proximity (mi)
0,Manhattan,2595,Skylit Midtown Castle,1,$95.00,0.39,94.0,94.0,94.0,10.0,...,40.75362,-73.98377,Entire home/apt,225,45,355,American Museum of Natural History,-73.973988,40.781324,1.550960
1,Manhattan,5099,Large Cozy 1 BR Apartment In Midtown East,2,$125.00,0.59,90.0,90.0,90.0,10.0,...,40.74767,-73.97500,Entire home/apt,200,74,129,American Museum of Natural History,-73.973988,40.781324,1.777736
2,Manhattan,5178,Large Furnished Room Near B'way,2,$15.00,3.53,84.0,84.0,84.0,9.0,...,40.76489,-73.98493,Private room,79,430,220,American Museum of Natural History,-73.973988,40.781324,1.041993
3,Manhattan,5238,Cute & Cozy Lower East Side 1 bdrm,3,$40.00,1.27,94.0,94.0,94.0,10.0,...,40.71344,-73.99037,Entire home/apt,150,160,188,American Museum of Natural History,-73.973988,40.781324,3.686422
4,Manhattan,5441,Central Manhattan/near Broadway,2,$0.00,1.58,97.0,97.0,97.0,10.0,...,40.76076,-73.98867,Private room,85,188,39,American Museum of Natural History,-73.973988,40.781324,1.333401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24873,Bronx,35800600,BEDROOM - SAFE ZONE - NO HIDDEN FEES!,2,$15.00,1.85,80.0,80.0,80.0,8.0,...,40.87852,-73.85003,Private room,36,2,47,Bronx Zoo,-73.876998,40.850595,2.049455
24874,Bronx,8265482,Cozy Private Room Near Bronx Zoo,2,$30.00,0.11,93.0,93.0,93.0,9.0,...,40.84328,-73.88173,Private room,79,3,179,Bronx Zoo,-73.876998,40.850595,0.459987
24875,Bronx,26004876,Ivoire Realty,1,$30.00,0.11,93.0,93.0,93.0,9.0,...,40.86259,-73.90359,Private room,65,2,0,Bronx Zoo,-73.876998,40.850595,1.540117
24876,Bronx,32475687,Two bedroom apartment close to subway,3,$40.00,3.13,97.0,97.0,97.0,10.0,...,40.84321,-73.83129,Entire home/apt,100,22,3,Bronx Zoo,-73.876998,40.850595,2.444646
